In [1]:
from fastai.tabular import *
from sklearn import preprocessing

In [2]:
data_path = Path("/home/jupyter/data/bimbo")

In [3]:
data_path.ls()

[PosixPath('/home/jupyter/data/bimbo/test.csv'),
 PosixPath('/home/jupyter/data/bimbo/zips'),
 PosixPath('/home/jupyter/data/bimbo/producto_tabla.csv'),
 PosixPath('/home/jupyter/data/bimbo/cliente_tabla.csv'),
 PosixPath('/home/jupyter/data/bimbo/models'),
 PosixPath('/home/jupyter/data/bimbo/sample_submission.csv'),
 PosixPath('/home/jupyter/data/bimbo/town_state.csv'),
 PosixPath('/home/jupyter/data/bimbo/tmp'),
 PosixPath('/home/jupyter/data/bimbo/train.csv')]

In [4]:
df_geo = pd.read_csv(data_path/'town_state.csv')
df_client = pd.read_csv(data_path/'cliente_tabla.csv')
df_product = pd.read_csv(data_path/'producto_tabla.csv')

os.makedirs('tmp', exist_ok=True)

In [5]:
if os.path.isfile('tmp/train_raw'):
    print("reading feather")
    df = pd.read_feather('tmp/train_raw')
else:
    print("reading train.csv")
    df = pd.read_csv(data_path/'train.csv')
    df.to_feather('tmp/train_raw')

reading feather


In [6]:
if os.path.isfile('tmp/test_raw'):
    print("reading feather")
    df_test = pd.read_feather('tmp/test_raw')
else:
    print("reading test.csv")
    df_test = pd.read_csv(data_path/'test.csv')
    df_test.to_feather('tmp/test_raw')

reading feather


In [7]:
df.sample(10)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
62198738,8,2234,1,1137,689716,1309,3,20.28,0,0.0,3
38386198,6,1632,1,1206,1095521,34868,10,30.20,0,0.0,10
22766210,5,1122,1,1471,2244626,972,2,37.96,0,0.0,2
10973888,3,22560,1,2821,1296220,5355,4,20.76,0,0.0,4
55741151,8,1243,1,1216,4600393,1250,5,38.20,0,0.0,5
2738595,3,1310,1,1253,91564,1250,4,30.56,0,0.0,4
2813869,3,1311,1,1283,2229298,1150,5,69.80,0,0.0,5
44187882,7,1152,4,6619,4619959,5380,3,25.26,0,0.0,3
21802922,4,22090,1,2102,1028163,30532,6,46.68,0,0.0,6
64260348,9,1121,1,1011,381749,32802,4,47.64,0,0.0,4


In [8]:
df_product.sample(10)

,Producto_ID,NombreProducto
170,2810,Panquecitos Gotas Panera 10p 700g BIM 2810
720,31781,Suavicremas Vainilla 158g SP TAB MLA 31781
1438,35802,Bran Frut con Avena 48g MTA BIM 35802
366,9756,Butter Milk Bread 680g ORO 9756
401,30235,Barritas Pina KC 8p 600g CU MLA 30235
446,30421,Silueta Fresa 6p 132g BIM 30421
1818,37153,Orejitas Panera 500g CR1 GBI 37153
2344,46171,Barritas Fresa 67g Prom TAB MLA 46171
588,31246,Bubulubu Twin Pack 70g Bolsa TR RIC 31246
647,31514,Polvorones Tubo 18p 250g SP MTB MLA 31514


In [9]:
df_client.sample(10)

,Cliente_ID,NombreCliente
189344,503759,ALICIA
798248,4581452,NO IDENTIFICADO
50459,121404,LOS PINOS
27840,70868,LONCHERIA LOS PONOS
371117,1395086,EMILIO LOPEZ VERA
134817,335013,ANDY
861779,4693458,NO IDENTIFICADO
530359,2175596,ROSALBA
176236,460919,CRISTIAN
602161,2430070,LONCHERIA ROSA


In [10]:
df_geo.sample(10)

,Agencia_ID,Town,State
777,25649,2564 MOCTEZUMA,SONORA
288,1669,2355 JALAPA I,VERACRUZ
471,2273,2386 COATZACOALCOS MLA.,VERACRUZ
382,2040,2181 ZIHUATANEJO,GUERRERO
12,1123,2094 CHALCO_BM,ESTADO DE MÉXICO
344,1977,2269 PUERTO VALLARTA BIMBO,JALISCO
37,1168,2092 AG. SANTA CLARA AUTOSERVICIOS,ESTADO DE MÉXICO
408,2081,2081 NAUCALPAN II,ESTADO DE MÉXICO
741,24689,2468 MATAMOROS BIMBO,TAMAULIPAS
297,1679,2056 PUEBLA NORTE MARINELA,PUEBLA


In [11]:
print(len((df_geo['Town'] + ", " + df_geo['State']).unique()))
print(len(df_geo))

261
790


In [12]:
print(len(df_client['NombreCliente'].unique()))
print(len(df_client))

311155
935362


In [13]:
print(len(df_product['NombreProducto'].unique()))
print(len(df_product))

2592
2592


In [14]:
df_geo['geo'] = (df_geo['Town'] + ", " + df_geo['State'])
le_geo = preprocessing.LabelEncoder()
df_geo['geo'] = le_geo.fit_transform(df_geo['geo'])
df_geo.to_feather('tmp/geo_map')

In [15]:
le_client = preprocessing.LabelEncoder()
df_client['NombreCliente'] = le_client.fit_transform(df_client['NombreCliente'])
df_client.to_feather('tmp/client_map')

In [16]:
le_product = preprocessing.LabelEncoder()
df_product['NombreProducto'] = le_product.fit_transform(df_product['NombreProducto'])
df_product.to_feather('tmp/product_map')

In [17]:
df = (df.merge(df_geo[['Agencia_ID', 'geo']], how='left', on='Agencia_ID')
      .merge(df_product, how='left', on='Producto_ID')
      .merge(df_client, how='left', on='Cliente_ID')
     )
df.to_feather('tmp/df_merge')
df.head()

In [18]:
f = 0.1
df_small = df.sample(frac=f)
df_small.reset_index().to_feather('tmp/df_merge_small')
# df_small.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,geo,NombreProducto,NombreCliente
72526790,9,2057,1,1265,3421848,1220,1,7.64,0,0.0,1,32,574,230616
23934384,5,1219,1,1604,889558,9217,2,18.00,2,18.0,0,24,622,164872
12810075,4,1215,1,1410,200854,1230,1,16.67,0,0.0,1,27,1633,180314
26532175,5,1441,1,2853,1946707,43069,2,14.82,0,0.0,2,215,1671,209805
12870532,4,1216,1,2126,441879,30572,6,37.50,0,0.0,6,4,116,269578
